In [2]:
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk
import cv2
import face_recognition
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
from PIL import Image, ImageTk
from PIL.Image import Resampling

# File to store registered users
REGISTERED_USERS_FILE = "registered_users.json"

# Load registered users from file (if it exists)
if os.path.exists(REGISTERED_USERS_FILE):
    with open(REGISTERED_USERS_FILE, "r") as file:
        registered_users = json.load(file)
else:
    registered_users = {}

# Directory to store registered faces
FACES_DIR = "registered_faces"
if not os.path.exists(FACES_DIR):
    os.makedirs(FACES_DIR)

# Dictionary to track the last notification time for each user
last_notification_time = {}

def save_registered_users():
    """Saves the registered users to a JSON file."""
    with open(REGISTERED_USERS_FILE, "w") as file:
        json.dump(registered_users, file)

def register_face(user_type):
    """Registers a new student or faculty face (with GUI input)."""
    name = simpledialog.askstring("Registration", f"Enter {user_type} Name:")
    if name is None or name == "":
        return

    user_id = simpledialog.askstring("Registration", f"Enter {user_type} ID:")
    if user_id is None or user_id == "":
        return

    if user_id in registered_users:
        messagebox.showinfo("Error", f"{user_type} with ID {user_id} is already registered.")
        return

    # Ask for branch and semester if the user is a student
    if user_type == "Student":
        branch = simpledialog.askstring("Registration", "Enter Branch (e.g., CSE, ECE, EE):")
        if branch is None or branch == "":
            return

        Semester = simpledialog.askstring("Registration", "Enter Semester (e.g., 1, 2, 3, 4):")
        if Semester is None or Semester == "":
            return

        # Store additional details for students
        registered_users[user_id] = {
            "name": name,
            "type": user_type,
            "branch": branch,
            "Semester": Semester
        }
    else:
        # For faculty, store only name and type
        registered_users[user_id] = {
            "name": name,
            "type": user_type
        }

    save_registered_users()

    video_capture = cv2.VideoCapture(0)

    while True:
        ret, frame = video_capture.read()
        if not ret:
            continue

        cv2.imshow("Face Registration", frame)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        elif key == ord('s'):
            face_encodings = face_recognition.face_encodings(frame)
            if face_encodings:
                face_encoding = face_encodings[0]

                image_path = os.path.join(FACES_DIR, f"{user_id}.jpg")
                cv2.imwrite(image_path, frame)

                encoding_path = os.path.join(FACES_DIR, f"{user_id}.npy")
                np.save(encoding_path, face_encoding)

                messagebox.showinfo("Success", f"{user_type} registered successfully!")
                video_capture.release()
                cv2.destroyAllWindows()
                return

    video_capture.release()
    cv2.destroyAllWindows()

def delete_user(user_type):
    """Deletes a student or faculty from the dataset."""
    user_id = simpledialog.askstring("Delete User", f"Enter {user_type} ID to delete:")
    if user_id is None or user_id == "":
        return

    if user_id not in registered_users:
        messagebox.showinfo("Error", f"{user_type} with ID {user_id} not found.")
        return

    del registered_users[user_id]
    save_registered_users()

    encoding_path = os.path.join(FACES_DIR, f"{user_id}.npy")
    image_path = os.path.join(FACES_DIR, f"{user_id}.jpg")

    if os.path.exists(encoding_path):
        os.remove(encoding_path)
    if os.path.exists(image_path):
        os.remove(image_path)

    messagebox.showinfo("Success", f"{user_type} with ID {user_id} deleted successfully.")

def mark_attendance(subject, faculty, Semester, branch):
    """Marks attendance based on face recognition and saves to subject-specific Excel files."""
    known_face_encodings = []
    known_face_ids = []

    for filename in os.listdir(FACES_DIR):
        if filename.endswith(".npy"):
            user_id = filename[:-4]
            encoding_path = os.path.join(FACES_DIR, filename)
            encoding = np.load(encoding_path)
            known_face_encodings.append(encoding)
            known_face_ids.append(user_id)

    video_capture = cv2.VideoCapture(0)

    while True:
        ret, frame = video_capture.read()
        if not ret:
            continue

        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            user_id = "Unknown"

            if True in matches:
                match_index = matches.index(True)
                user_id = known_face_ids[match_index]
                user_info = registered_users.get(user_id, {})
                name = user_info.get("name", "Unknown")

                now = datetime.now()
                date_str = now.strftime("%d-%m-%y")
                time_str = now.strftime("%H:%M:%S")

                last_notification = last_notification_time.get(user_id)
                if last_notification and (now - last_notification).total_seconds() < 3600:
                    continue

                end_time = now + timedelta(hours=1)
                end_time_str = end_time.strftime("%H:%M:%S")

                last_notification_time[user_id] = now

                attendance_file = f"attendance_{branch}_{Semester}_{subject}_{date_str}.xlsx"

                if os.path.exists(attendance_file):
                    df = pd.read_excel(attendance_file)
                else:
                    df = pd.DataFrame(columns=["Student ID", "Name", "Subject", "Faculty", "Semester", "Branch", "Date", "Time"])

                new_row = {
                    "Student ID": user_id,
                    "Name": name,
                    "Subject": subject,
                    "Faculty": faculty,
                    "Semester": Semester,
                    "Branch": branch,
                    "Date": date_str,
                    "Time": f"{time_str} - {end_time_str}"
                }
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
                df.to_excel(attendance_file, index=False)
                print(f"Attendance marked for {name} in {subject} with {faculty} from {time_str} to {end_time_str}")

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)

        cv2.imshow("Attendance", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

def mark_attendance_gui():
    """GUI function to mark attendance based on manually entered details."""
    branch = branch_entry.get()
    Semester = Semester_entry.get()
    subject = subject_entry.get()
    faculty = faculty_entry.get()

    if branch and Semester and subject and faculty:
        mark_attendance(subject, faculty, Semester, branch)
    else:
        messagebox.showinfo("Error", "Please enter branch, year, subject, and faculty.")

# GUI Setup
root = tk.Tk()
root.title("Attendance Management")
root.geometry("500x600")
root.configure(bg="#f0f0f0")



# Frame for buttons
button_frame = tk.Frame(root, bg="#f0f0f0")
button_frame.pack(pady=(120,20))

# Buttons
register_student_button = tk.Button(
    button_frame, 
    text="Register Student", 
    command=lambda: register_face("Student"),  # Use lambda to pass arguments
    bg="#4CAF50", 
    fg="white", 
    font=("Arial", 12))
register_student_button.grid(row=0, column=0, padx=10, pady=10)

register_faculty_button = tk.Button(
    button_frame, 
    text="Register Faculty", 
    command=lambda: register_face("Faculty"),  # Use lambda to pass arguments
    bg="#2196F3", 
    fg="white", 
    font=("Arial", 12))
register_faculty_button.grid(row=0, column=1, padx=10, pady=10)

delete_student_button = tk.Button(
    button_frame, 
    text="Delete Student", 
    command=lambda: delete_user("Student"),  # Use lambda to pass arguments
    bg="#f44336", 
    fg="white", 
    font=("Arial", 12))
delete_student_button.grid(row=1, column=0, padx=10, pady=10)

delete_faculty_button = tk.Button(
    button_frame, 
    text="Delete Faculty", 
    command=lambda: delete_user("Faculty"),  # Use lambda to pass arguments
    bg="#FF9800", 
    fg="white", 
    font=("Arial", 12))
delete_faculty_button.grid(row=1, column=1, padx=10, pady=10)

# Frame for attendance
attendance_frame = tk.Frame(root, bg="#f0f0f0")
attendance_frame.pack(pady=(20,20))

branch_label = tk.Label(attendance_frame, text="Enter Branch:", bg="#f0f0f0", font=("Arial", 12))
branch_label.grid(row=0, column=0, padx=10, pady=5)

branch_entry = ttk.Entry(attendance_frame, font=("Arial", 12))
branch_entry.grid(row=0, column=1, padx=10, pady=5)

Semester_label = tk.Label(attendance_frame, text="Enter Semester:", bg="#f0f0f0", font=("Arial", 12))
Semester_label.grid(row=1, column=0, padx=10, pady=5)

Semester_entry = ttk.Entry(attendance_frame, font=("Arial", 12))
Semester_entry.grid(row=1, column=1, padx=10, pady=5)

subject_label = tk.Label(attendance_frame, text="Enter Subject:", bg="#f0f0f0", font=("Arial", 12))
subject_label.grid(row=2, column=0, padx=10, pady=5)

subject_entry = ttk.Entry(attendance_frame, font=("Arial", 12))
subject_entry.grid(row=2, column=1, padx=10, pady=5)

faculty_label = tk.Label(attendance_frame, text="Enter Faculty:", bg="#f0f0f0", font=("Arial", 12))
faculty_label.grid(row=3, column=0, padx=10, pady=5)

faculty_entry = ttk.Entry(attendance_frame, font=("Arial", 12))
faculty_entry.grid(row=3, column=1, padx=10, pady=5)

mark_attendance_button = tk.Button(attendance_frame, text="Mark Attendance", command=mark_attendance_gui, bg="#9C27B0", fg="white", font=("Arial", 12))
mark_attendance_button.grid(row=4, column=0, columnspan=2, pady=10)

def on_closing():
    root.destroy()
    cv2.destroyAllWindows()

root.protocol("WM_DELETE_WINDOW", on_closing)

root.mainloop()